In [1]:
import pandas as pd
import numpy as np
import os
import re, string, timeit
import nltk
import sys
sys.path.append('/workspace/Translation_lancet/')
from utils_lancet import *
from nltk.corpus import stopwords
from multiprocessing import Pool
import tqdm
import time
import re
import multiprocessing
import pyreadr

folder = "Multiglobal_result_2023/"# "English_2023/" #  "EU_result/" # 

In [2]:
#with open("EU_result/corpus_tokens.txt", "r") as fp:
 #   corpus = json.load(fp)

In [2]:
df_kwic = pd.read_pickle(folder + 'df_kwic_health.pkl')

In [3]:
climate_dict = ["climate_change", "changing_climate", "climate_emergency", "climate_crisis", 
                "climate_decay", "global_warming","green_house", "temperature", "extreme_weather",
                "global_environmental_change", "climate_variability", "greenhouse", 
                "greenhouse-gas", "low_carbon", "ghge", "ghges", "renewable_energy", 
                "carbon_emission", "carbon_emissions", "carbon_dioxide", "carbon-dioxide",
                "co2_emission", "co2_emissions", "climate_pollutant", "climate_pollutants", 
                "decarbonization", "decarbonisation", "carbon_neutral", "carbon-neutral", 
                "carbon_neutrality", "climate_neutrality", "climate_action", "net-zero", "net_zero", 'carbonemission', "carbonemissions"]

compoundlist = ["air pollution", "mental disorder", "mental disorders", "climate change", "changing climate", "climate emergency", 
          "climate crisis", "climate decay", "global warming", "green house", "extreme weather", "global environmental change", 
          "climate variability",  "low carbon", "renewable energy", "carbon emission", "carbon emissions", "carbon dioxide", 
          "co2 emission", "co2 emissions", "climate pollutant", "climate pollutants", "carbon neutral", "carbon neutrality", 
          "climate neutrality", "climate action", "net zero", "covid 19", "corona virus", "sars cov 2"]


def kwic_df_search(row, search_list = climate_dict,):
    _, row = row
    corpus = " ".join ([str(row["left"]), str(row["right"])])
    kwic_health_climate = KWIC(search_list = search_list, corpus = corpus.split(), id_ = row.Id, compound_list = compoundlist, )
    kwic_df, _ = kwic_health_climate.kwic_search(window = 25,)
    kwic_df["health_key"] = row.key_word 
    kwic_df["old_index"] = row.token_index 
    return kwic_df
    
test = False
res = []

start = 0
cores = multiprocessing.cpu_count()
for df in np.array_split(df_kwic, 5):
    if __name__ == '__main__':
        with Pool(cores) as p:
            if test == True:
                df = df.iloc[:5]
            r = list(tqdm.tqdm(p.imap(kwic_df_search, df.iterrows()), total = len(df)))
            res.append(r)
            start=+1
            #break

100%|██████████| 214573/214573 [03:20<00:00, 1069.27it/s]


In [4]:
r_concat = [s for i in res for s in i]

In [5]:
res = pd.concat(r_concat)  

In [6]:
res.to_csv(folder + 'df_kwic_health_climate.csv')

In [7]:
intersection = pd.read_csv(folder + 'df_kwic_health_climate.csv')

In [8]:
# corss gender and covid

# covid dictionary
covid_dict = ["covid-19", "covid19", "covid_19", "corona", "corona_virus", "sars-cov-2", "covid", "coronavirus"]

# gender dictionary
gender_dict = ["gender", "male", "female", "man", "men", "woman", "women", "sex"]

In [9]:
def kwic_df_search(row, search_list = gender_dict,):
    _, row = row
    corpus = " ".join ([str(row["left"]), str(row["right"])])
    kwic_health_climate = KWIC(search_list = search_list, corpus = corpus.split(), id_ = row.Id, compound_list = compoundlist, )
    kwic_df, _ = kwic_health_climate.kwic_search(window = 25,)
    kwic_df["climate_key"] = row.key_word 
    kwic_df["health_key"] = row.health_key
    kwic_df["old_index"] = row.token_index 
    return kwic_df

test = False

cores = multiprocessing.cpu_count()
df = intersection[["left", "right", "Id", "old_index", "health_key", "key_word", "token_index"]]
if __name__ == '__main__':
    with Pool(cores) as p:
        if test == True:
            df = df.iloc[:500]
        r = list(tqdm.tqdm(p.imap(kwic_df_search, df.iterrows()), total = len(df)))
        res = pd.concat(r)  

100%|██████████| 40995/40995 [00:40<00:00, 1018.79it/s]


In [10]:
res.to_csv(folder + "gender.csv", index = False)
#res.to_csv(folder + "covid.csv", index = False)

In [11]:
def kwic_df_search(row, search_list = covid_dict,):
    _, row = row
    corpus = " ".join ([str(row["left"]), str(row["right"])])
    kwic_health_climate = KWIC(search_list = search_list, corpus = corpus.split(), id_ = row.Id, compound_list = compoundlist, )
    kwic_df, _ = kwic_health_climate.kwic_search(window = 25,)
    kwic_df["climate_key"] = row.key_word 
    kwic_df["health_key"] = row.health_key
    kwic_df["old_index"] = row.token_index 
    return kwic_df

test = False

cores = multiprocessing.cpu_count()
df = intersection[["left", "right", "Id", "old_index", "health_key", "key_word", "token_index"]]
if __name__ == '__main__':
    with Pool(cores) as p:
        if test == True:
            df = df.iloc[:500]
        r = list(tqdm.tqdm(p.imap(kwic_df_search, df.iterrows()), total = len(df)))
        res = pd.concat(r)  

100%|██████████| 40995/40995 [00:41<00:00, 993.05it/s] 


In [12]:
#res.to_csv(folder + "gender.csv", index = False)
res.to_csv(folder + "covid.csv", index = False)

In [13]:
res

,key_word,left,right,left_check,right_check,token_index,Id,climate_key,health_key,old_index
0,covid-19,role managing executive assessing risks opport...,goal • reorganizing quarantine system due prol...,system prevent,goal •,15.0,478191.0,climate_change,pandemic,17.0
0,covid-19,prevent,contagion climate_change goal • reorganizing q...,prevent,contagion climate_change,2.0,478191.0,climate_change,pandemic,39.0
0,covid-19,lead dramatic shifts interoperable data analyt...,pandemic models future health medical,year conflict,pandemic models,27.0,478078.0,climate_change,diseases,41.0
0,covid-19,control mosquito-spread diseases potential geo...,pandemic models medical science also undermini...,year conflict,pandemic models,20.0,478078.0,climate_change,health,19.0
0,covid-19,millions year conflict,pandemic models future health medical science ...,year conflict,pandemic models,4.0,478078.0,climate_change,health,3.0
...,...,...,...,...,...,...,...,...,...,...
0,corona,still,disaster asked many schools needed able make s...,still,disaster asked,2.0,473700.0,global_warming,infection,39.0
0,corona,,disaster asked many schools needed able make s...,,disaster asked,1.0,473700.0,global_warming,infection,40.0
0,corona,corona disaster asked many schools needed able...,disaster asked many schools needed able make s...,corona disaster asked many schools needed able...,disaster asked,0.0,473700.0,global_warming,infection,41.0
0,corona_virus,make stronger going make stronger im going try...,rich new people able see heavy workers prepare...,spread river,rich new,26.0,474209.0,greenhouse,infection,17.0
